In [1]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.8 MB/s eta 0:00:00


In [2]:
from google.colab import files
import os

# Upload the zip file
uploaded = files.upload()

Saving I01-W01.pdf to I01-W01.pdf
Saving I01-W02.pdf to I01-W02.pdf
Saving I01-W03.pdf to I01-W03.pdf
Saving I01-W04.pdf to I01-W04.pdf
Saving I02-W01.pdf to I02-W01.pdf
Saving I03-W01.pdf to I03-W01.pdf
Saving I06-W01.pdf to I06-W01.pdf
Saving I07-W01.pdf to I07-W01.pdf
Saving I07-W02.pdf to I07-W02.pdf
Saving I07-W03.pdf to I07-W03.pdf
Saving I07-W04.pdf to I07-W04.pdf
Saving I08-W01.pdf to I08-W01.pdf
Saving I09-W01.pdf to I09-W01.pdf
Saving I09-W02.pdf to I09-W02.pdf
Saving I09-W03.pdf to I09-W03.pdf
Saving I09-W04.pdf to I09-W04.pdf
Saving I10-W01.pdf to I10-W01.pdf
Saving I10-W02.pdf to I10-W02.pdf
Saving I10-W03.pdf to I10-W03.pdf
Saving I10-W04.pdf to I10-W04.pdf
Saving I12-W01.pdf to I12-W01.pdf
Saving I12-W02.pdf to I12-W02.pdf
Saving I15-W02.pdf to I15-W02.pdf
Saving I16-W01.pdf to I16-W01.pdf
Saving I17-W01.pdf to I17-W01.pdf
Saving I17-W02.pdf to I17-W02.pdf
Saving I17-W03.pdf to I17-W03.pdf
Saving I17-W04.pdf to I17-W04.pdf
Saving I18-W01.pdf to I18-W01.pdf
Saving I18-W02

In [8]:
from collections import namedtuple
import pandas as pd
import pdfplumber
import re
Output = pd.DataFrame(columns= ['Analysis_Year','Jurisdiction', 'Time_Analyzed', 'LOS',  'Density'] )

In [9]:
import re
import pdfplumber
import pandas as pd

# Initialize a list to store extracted data
data_list = []

for filename in uploaded.keys():
    all_text = []

    # Extract text from PDF
    with pdfplumber.open(filename) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                all_text.append(f"Page {page_num}:\n{page_text}\n")

    # Combine all text
    full_text = "\n".join(all_text)

    # Define refined patterns to extract specific information
    year_pattern = r"Analysis Year \s*(.+)"  # Extracts everything after year until newline
    jurisdiction_pattern = r"Jurisdiction\s+(\S+)"  # Extracts single-word jurisdiction
    time_analyzed_pattern = r"Time Analyzed\s*(.+)"  # Extracts words after 'Time Analyzed'
    los_pattern = r"Level of Service \(LOS\)\s+([A-F])"  # Extracts single LOS letter
    density_pattern = r"Density in Ramp Influence Area \(DR\), pc/km/\s+([\d\.]+)"  # Extracts numeric density
    density_pattern_W1 = r"Density \(D\), pc/km/ln\s+([\d\.]+)"

    # Initialize variables
    site_number = None
    jurisdiction = None
    time_analyzed = None
    level_of_service = None
    density = None

    # Search for patterns in the text
    year_match = re.search(year_pattern, full_text)
    jurisdiction_match = re.search(jurisdiction_pattern, full_text)
    time_analyzed_match = re.search(time_analyzed_pattern, full_text)
    los_match = re.search(los_pattern, full_text)


    # Extract data from matches
    if year_match:
        site_number = year_match.group(1).strip()

    if jurisdiction_match:
        jurisdiction = jurisdiction_match.group(1).strip()

    if time_analyzed_match:
        time_analyzed = time_analyzed_match.group(1).strip()

    if los_match:
        level_of_service = los_match.group(1).strip()
    if "W" in jurisdiction:
        density_match = re.search(density_pattern_W1, full_text)
    else:
        density_match = re.search(density_pattern, full_text)
    if density_match:
        density = density_match.group(1).strip()

    # Store the extracted values in a dictionary
    extracted_data = {
        "Site Number After Analysis Year": site_number,
        "Jurisdiction": jurisdiction,
        "Time Analyzed": time_analyzed,
        "Level of Service (LOS)": level_of_service,
        "Density in Ramp Influence Area (DR), pc/km/ln": density
    }

    # Append the dictionary to the list
    data_list.append(extracted_data)

# Convert list to Pandas DataFrame
output = pd.DataFrame(data_list)

In [10]:
output.head()

,Site Number After Analysis Year,Jurisdiction,Time Analyzed,Level of Service (LOS),"Density in Ramp Influence Area (DR), pc/km/ln"
0,2040,I01-W01,AM,B,9.7
1,2040,I01-W02,AM,A,4.7
2,2040,I01-W03,AM,B,8.1
3,2040,I01-W04,AM,A,4.6
4,2040,I02-W01,AM,B,8.1


In [ ]:
output.to_excel('HCS_Report.xlsx', index=False)